<a href="https://www.kaggle.com/code/lyhaiyen/ratecreditscore?scriptVersionId=112770269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # xử lí dữ liệu, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing # để chuẩn hóa, scale data
from sklearn.preprocessing import StandardScaler
import os, datetime, sys, random, time
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn import model_selection # Chia mảng hoặc ma trận thành tập con test và train ngẫu nhiên
from sklearn.tree import export_graphviz
import scikitplot as skplt
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import tree # Decision Tree
from sklearn import linear_model # https://scikit-learn.org/stable/modules/linear_model.html
from sklearn.feature_selection import SelectFromModel
%matplotlib inline
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # https://matplotlib.org/stable/tutorials/intermediate/gridspec.html
from mlxtend import classifier # machine learning extensions cho data science
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
from scipy import stats, special # hỗ trợ các công thức về thống kê
import shap
import catboost as ctb

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from IPython.display import HTML

# ĐỌC DỮ LIỆU

In [ ]:
train=pd.read_csv("datafile/cs-training.csv")
finalTrain = train.copy()
trainIndex = finalTrain['Unnamed: 0']
finalTrain.drop(train.columns.values[0],axis=1, inplace = True)
print('Số lượng mẫu tập huấn luyện = %d' %(train.shape[0]))
print('Số lượng thuộc tính tập huấn luyện = %d' %(train.shape[1]))
test=pd.read_csv("datafile/cs-test.csv")
finalTest = test.copy()
testIndex = finalTest['Unnamed: 0']
finalTest.drop(test.columns.values[0], axis=1, inplace = True)
print('Số lượng mẫu tập kiểm tra = %d' %(test.shape[0]))
print('Số lượng thuộc tính tập kiểm tra = %d' %(test.shape[1]))

Bỏ cột unnamed: 0 vì đấy là index trong 2 tập và không quan trọng trong việc dự đoán mô hình

# Khai phá dữ liệu

### Tập train

In [ ]:
finalTrain.head()

In [ ]:
finalTrain.info()
#finalTrain.dtypes

dữ liệu ở dưới dạng int và float, và feature MonthlyIncome và NumberOfDependents có dữ liệu bị thiếu

In [ ]:
finalTrain.describe()

### Tập Test

In [ ]:
finalTest.head()

In [ ]:
finalTest.info()

In [ ]:
finalTest.describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
finalTrain['SeriousDlqin2yrs'].value_counts().plot.pie(explode=[0,0.1], autopct='%1.1f%%', ax=axes[0])
axes[0].set_title('SeriousDlqin2yrs')
sns.countplot('SeriousDlqin2yrs',data=finalTrain,ax=axes[1])
axes[1].set_title('Số lượng người') #Người đã trải qua vi phạm 90 ngày quá hạn hoặc tệ hơn
plt.show()

In [ ]:
train['SeriousDlqin2yrs'].value_counts()

Tỷ lệ của các trường hợp ngoại lệ 0 và 1 được tìm thấy là 93,3% đến 6,7%, xấp xỉ tỷ lệ 14: 1. Do đó, tập dữ liệu rất mất cân đối. Không thể dựa vào điểm chính xác để dự đoán thành công mô hình

In [ ]:
fig = plt.figure(figsize=[40,40])
for col,i in zip(finalTrain.columns,range(1,13)):
    axes = fig.add_subplot(7,2,i)
    sns.regplot(finalTrain[col],finalTrain.SeriousDlqin2yrs,ax=axes) #để vẽ nhiều mối quan hệ tuyến tính trong một tập dữ liệu
plt.show()

Trong các cột NumberOfTime30-59DaysPastDueNotWorse, NumberOfTime60-89DaysPastDueNotWorse và NumberOfTimes90DaysLate, có phạm vi quá hạn vượt quá 90 phổ biến trên cả 3 feature.
Có một số giá trị cao bất thường cho DebtRatio và RevolvingUtilizationOfUnsecuredLines.
Những feature trên có giá trị ngoại lai

In [ ]:
print("Những giá trị duy nhất trong những giá trị '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTrain[finalTrain['NumberOfTime30-59DaysPastDueNotWorse']>=90] ['NumberOfTime30-59DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '60-89 ngày' khi '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTrain[finalTrain['NumberOfTime60-89DaysPastDueNotWorse']>=90] ['NumberOfTime60-89DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '90 ngày' khi '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTrain[finalTrain['NumberOfTimes90DaysLate']>=90] ['NumberOfTimes90DaysLate']))
print("Những giá trị duy nhất trong những giá trị '60-89 ngày' khi '30-59 ngày' ít hơn 90: " ,
      np.unique(finalTrain[finalTrain['NumberOfTime60-89DaysPastDueNotWorse']<90] ['NumberOfTime60-89DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '90 ngày' khi '30-59 ngày' ít hơn 90: " ,
      np.unique(finalTrain[finalTrain['NumberOfTimes90DaysLate']<90]['NumberOfTimes90DaysLate']))
print("Tỷ lệ của lớp 1 với những giá trị đặc biệt 96,98: ", 
      round(finalTrain[finalTrain['NumberOfTime30-59DaysPastDueNotWorse']>=90]['SeriousDlqin2yrs'].sum()*100/
        len(finalTrain[finalTrain['NumberOfTime30-59DaysPastDueNotWorse']>=90]['SeriousDlqin2yrs']),2),'%')

Có thể thấy điều sau rằng khi các bản ghi trong cột 'NumberOfTime30-59DaysPastDueNotWorse' lớn hơn 90, các cột khác ghi lại số lần các khoản thanh toán quá hạn X ngày cũng có cùng giá trị. Chúng tôi sẽ phân loại những nhãn này là nhãn đặc biệt vì tỷ lệ lớp cao bất thường ở mức 54,65%.

Các giá trị 96 và 98 này có thể được xem là lỗi kế toán. Do đó, sẽ thay thế chúng bằng giá trị lớn nhất trước 96, tức là 13, 11 và 17, hoặc là loại bỏ nó

In [ ]:
finalTrain.loc[finalTrain['NumberOfTime30-59DaysPastDueNotWorse'] >= 90, 'NumberOfTime30-59DaysPastDueNotWorse'] = 13
finalTrain.loc[finalTrain['NumberOfTime60-89DaysPastDueNotWorse'] >= 90, 'NumberOfTime60-89DaysPastDueNotWorse'] = 11
finalTrain.loc[finalTrain['NumberOfTimes90DaysLate'] >= 90, 'NumberOfTimes90DaysLate'] = 17
print("Các giá trị duy nhất trong 30-59Days", np.unique(finalTrain['NumberOfTime30-59DaysPastDueNotWorse']))
print("Các giá trị duy nhất trong 60-89Days", np.unique(finalTrain['NumberOfTime60-89DaysPastDueNotWorse']))
print("các giá trị duy nhất trong 90Days", np.unique(finalTrain['NumberOfTimes90DaysLate']))

#### Trên tập test

In [ ]:
print("Những giá trị duy nhất trong những giá trị '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTest[finalTest['NumberOfTime30-59DaysPastDueNotWorse']>=90] ['NumberOfTime30-59DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '60-89 ngày' khi '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTest[finalTest['NumberOfTime60-89DaysPastDueNotWorse']>=90] ['NumberOfTime60-89DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '90 ngày' khi '30-59 ngày' nhiều hơn hoặc bằng 90:", 
      np.unique(finalTest[finalTest['NumberOfTimes90DaysLate']>=90] ['NumberOfTimes90DaysLate']))
print("Những giá trị duy nhất trong những giá trị '30-59 ngày' ít hơn 90: " ,
      np.unique(finalTest[finalTest['NumberOfTime30-59DaysPastDueNotWorse']<90] ['NumberOfTime30-59DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '60-89 ngày' khi '30-59 ngày' ít hơn 90: " ,
      np.unique(finalTest[finalTest['NumberOfTime60-89DaysPastDueNotWorse']<90] ['NumberOfTime60-89DaysPastDueNotWorse']))
print("Những giá trị duy nhất trong những giá trị '90 ngày' khi '30-59 ngày' ít hơn 90: " ,
      np.unique(finalTest[finalTest['NumberOfTimes90DaysLate']<90]['NumberOfTimes90DaysLate']))

In [ ]:
finalTest.loc[finalTest['NumberOfTime30-59DaysPastDueNotWorse'] >= 90, 'NumberOfTime30-59DaysPastDueNotWorse'] = 19
finalTest.loc[finalTest['NumberOfTime60-89DaysPastDueNotWorse'] >= 90, 'NumberOfTime60-89DaysPastDueNotWorse'] = 9
finalTest.loc[finalTest['NumberOfTimes90DaysLate'] >= 90, 'NumberOfTimes90DaysLate'] = 18
print("Các giá trị duy nhất trong 30-59Days", np.unique(finalTest['NumberOfTime30-59DaysPastDueNotWorse']))
print("Các giá trị duy nhất trong 60-89Days", np.unique(finalTest['NumberOfTime60-89DaysPastDueNotWorse']))
print("các giá trị duy nhất trong 90Days", np.unique(finalTest['NumberOfTimes90DaysLate']))

### Tập Train

In [ ]:
print('Debt Ratio: \n',finalTrain['DebtRatio'].describe())
print('\nRevolving Utilization of Unsecured Lines: \n',finalTrain['RevolvingUtilizationOfUnsecuredLines'].describe())

In [ ]:
quantiles = [0.75,0.8,0.81,0.85,0.9,0.95,0.975,0.99]

for i in quantiles:
    print(i*100,'% quantile of debt ratio is: ',finalTrain.DebtRatio.quantile(i))

Quantile bắt đầu tăng nhanh từ 81%, vì vậy sẽ phải kiểm tra outlier từ quantile 81%, nhưng vì dữ liệu chỉ có 150000 mẫu nên sẽ xem xét từ quantile từ 95% và 97.5% để phân tích

In [ ]:
finalTrain[finalTrain['DebtRatio'] >= finalTrain['DebtRatio'].quantile(0.95)][['SeriousDlqin2yrs','MonthlyIncome']].describe()

Ở đây quan sát được:
Trong số 7501 khách hàng có tỷ lệ nợ lớn hơn 95%, tức là số lần nợ cao hơn thu nhập của họ, chỉ có 379 khách hàng có giá trị Thu nhập hàng tháng.
Giá trị Tối đa cho Thu nhập Hàng tháng là 1 và Tối thiểu là 0, lỗi nhập dữ liệu ?. Hãy kiểm tra xem liệu giá trị Thời gian trễ hạn nghiêm trọng trong 2 năm(SeriousDlqin2yrs) và Thu nhập hàng tháng(MonthlyIncome) có bằng nhau hay không.

In [ ]:
finalTrain[(finalTrain["DebtRatio"] > finalTrain["DebtRatio"].quantile(0.95)) & (finalTrain['SeriousDlqin2yrs'] == finalTrain['MonthlyIncome'])]

Do đó, nghi ngờ là đúng và có 331 trong số 379 hàng có Thu nhập hàng tháng bằng với Vi phạm nghiêm trọng trong 2 năm. Do đó, sẽ loại bỏ 331 giá trị ngoại lệ này khỏi phân tích của mình vì các giá trị hiện tại của chúng không hữu ích cho mô hình dự đoán  và sẽ thêm vào độ chệch và phương sai.
Lý do đằng sau điều này, là có 331 hàng trong đó tỷ lệ nợ lớn so với thu nhập của khách hàng và chúng không được xem xét kỹ lưỡng về việc mặc định không phải là lỗi nhập dữ liệu.

In [ ]:
finalTrain = finalTrain[-((finalTrain["DebtRatio"] > finalTrain["DebtRatio"].quantile(0.95)) & (finalTrain['SeriousDlqin2yrs'] == finalTrain['MonthlyIncome']))]
finalTrain # đã loại bỏ 331 khách hàng

### RevolvingUtilizationOfUnsecuredLines

Trường này về cơ bản thể hiện tỷ lệ số tiền nợ theo hạn mức tín dụng của khách hàng. 
Tỷ lệ cao hơn 1 được coi là một sai số nghiêm trọng. Tỷ lệ bằng 10 về mặt cơ bản cũng có vẻ khả thi, hãy xem có bao nhiêu 
khách hàng trong số này có Revolving Utilization of Unsecured Lines lớn hơn 10.

In [ ]:
finalTrain[finalTrain['RevolvingUtilizationOfUnsecuredLines']>10].describe()

Kiểm tra xem những giá trị của feature sử dụng quay vòng của hạn mức tín dụng không có bảo đảm nào làm ảnh hưởng ít hay nhiều đến feature thời gian trễ hạn nghiêm trọng trong 2 năm bằng cách tính giá tri trung bình của feature thời gian trễ hạn nghiêm trọng trong 2 năm theo giá trị của feature sử dụng quay vòng của hạn mức tín dụng không có bảo đảm.

In [ ]:
default_prop = []
for i in range(int(finalTrain["RevolvingUtilizationOfUnsecuredLines"].max())):
    temp_ = finalTrain.loc[finalTrain["RevolvingUtilizationOfUnsecuredLines"] >= i]
    default_prop.append([i, temp_["SeriousDlqin2yrs"].mean()])
default_prop

In [ ]:
sns.lineplot(x=[i[0] for i in default_prop], y=[i[1] for i in default_prop])
plt.title("Tỷ lệ người vi phạm(SeriousDlqin2yrs) khi sử dụng quay vòng của hạn mức tín dụng tối thiểu tăng lên")

In [ ]:
print("Tỷ lệ người vi phạm(SeriousDlqin2yrs) có Tổng số tiền sở hữu không vượt quá tổng hạn mức tín dụng: {}"\
     .format(finalTrain.loc[(finalTrain["RevolvingUtilizationOfUnsecuredLines"] >= 0) & (finalTrain["RevolvingUtilizationOfUnsecuredLines"] <= 1)]["SeriousDlqin2yrs"].mean()))

In [ ]:
print("Tỷ lệ người vi phạm(SeriousDlqin2yrs) có tổng số tiền sở hữu không vượt quá hoặc bằng 13 lần tổng hạn mức tín dụng:\n{}"\
     .format(finalTrain.loc[(finalTrain["RevolvingUtilizationOfUnsecuredLines"] >= 0) & (finalTrain["RevolvingUtilizationOfUnsecuredLines"] < 13)]["SeriousDlqin2yrs"].mean()))

In [ ]:
finalTrain[finalTrain['RevolvingUtilizationOfUnsecuredLines']>13].describe()

In [ ]:
finalTrain = finalTrain[finalTrain['RevolvingUtilizationOfUnsecuredLines']<=13]
finalTrain

Loại bỏ những khách hàng có Sử dụng quay vòng của hạn mức tín dụng không có bảo đảm lớn hơn 13. Và dữ liệu còn lại 149431 mẫu

## Xử Lí Missing và Null data

Vì MonthlyIncome là một giá trị số nguyên, chúng tôi sẽ thay thế các giá trị null bằng các giá trị trung vị.
Số người phụ thuộc có thể được mô tả như một biến phân loại, do đó nếu khách hàng có NaN cho số người phụ thuộc, điều đó có nghĩa là họ không có bất kỳ người phụ thuộc nào. Do đó, chúng tôi điền chúng bằng các số không

In [ ]:
def missingvalues(data):
    for col in data.columns:
        print('\t%s: %d' % (col,data[col].isna().sum()))
print('Số lượng các giá trị bị mất trong tập train:')
missingvalues(finalTrain)
print('Số lượng các giá trị bị mất trong tập test:')
missingvalues(finalTest)

In [ ]:
def MissingHandler(data):
    DataMissing = data.isnull().sum()*100/len(data)
    DataMissingByColumn = pd.DataFrame({'Tỷ lệ phần trăm dữ liệu NaN':DataMissing})
    DataMissingByColumn.sort_values(by='Tỷ lệ phần trăm dữ liệu NaN',ascending=False,inplace=True)
    return DataMissingByColumn[DataMissingByColumn['Tỷ lệ phần trăm dữ liệu NaN']>0]
MissingHandler(finalTrain)

In [ ]:
def replaceNaN(data):
    data["MonthlyIncome"].fillna(data["MonthlyIncome"].median(), inplace = True)
    data["NumberOfDependents"].fillna(0, inplace = True)
    return data
replaceNaN(finalTrain)

In [ ]:
missingvalues(finalTrain)

In [ ]:
MissingHandler(finalTest)

In [ ]:
replaceNaN(finalTest)# trừ SeriousDlqin2yrs

In [ ]:
missingvalues(finalTest)

## MA TRẬN TƯƠNG QUAN(CORRELATION MATRIX)

In [ ]:
fig = plt.figure(figsize = [10,7])
mask = np.zeros_like(finalTrain.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(finalTrain.corr(), cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9), mask = mask, annot=True, center = 0)
plt.title("Correlation Matrix (HeatMap)", fontsize = 15)

Từ heatmap tương quan ở trên, chúng ta có thể thấy các giá trị tương quan nhất vói SeriousDlqin2yrs là NumberOfTime30-59DaysPastDueNotWorse , NumberOfTime60-89DaysPastDueNotWorse và NumberOfTimes90DaysLate, RevolvingUtilizationOfUnsecuredLines

## Feature Engineering

Kết hợp tập train và tập test để thêm các tính năng trên cả dữ liệu và tiến hành phân tích sâu hơn. Sẽ chia chúng sau trước khi Kiểm tra mô hình.

In [ ]:
SeriousDlqIn2Yrs01 = finalTrain['SeriousDlqin2yrs']
SeriousDlqIn2Yrs02 = finalTest['SeriousDlqin2yrs']
finalTrain.drop('SeriousDlqin2yrs', axis = 1 , inplace = True)
finalTest.drop('SeriousDlqin2yrs', axis = 1 , inplace = True)

In [ ]:
finalData = pd.concat([finalTrain, finalTest])

finalData.shape

Thêm vào một số feature
MonthlyIncomePerPerson: Thu nhập hàng tháng chia cho số người phụ thuộc

MonthlyDebt: Thu nhập hàng tháng nhân với Tỷ lệ Nợ(Debt Ratio)

isRetired: Người có thu nhập hàng tháng bằng 0 và tuổi lớn hơn 65 (Tuổi nghỉ hưu)

RevolvingLines: Sự khác biệt giữa Số lượng Dòng Tín dụng Mở và Khoản cho vay(Number of Open Credit Lines and Loans) và Số Dòng Bất động sản và Khoản cho vay(Number of Real Estate Lines and Loans)

hasRevolvingLines: Nếu RevolvingLines tồn tại thì bằng 1 nếu không thì bằng 0

hasMultipleRealEstates: Nếu Số lượng Bất động sản lớn hơn 2

incomeDivByThousand:Thu nhập hàng tháng chia cho 1000. Có thể có nhiều khả năng là gian lận hoặc nó có thể là dấu hiệu người đó là trong một công việc mới và chưa có một tăng phần trăm lương được nêu ra. Cả hai nhóm đều báo hiệu nguy cơ cao hơn.

In [ ]:
finalData['MonthlyIncomePerPerson'] = finalData['MonthlyIncome']/(finalData['NumberOfDependents']+1)
finalData['MonthlyIncomePerPerson'].fillna(0, inplace=True)

finalData['MonthlyDebt'] = finalData['MonthlyIncome']*finalData['DebtRatio']
finalData['MonthlyDebt'].fillna(finalData['DebtRatio'],inplace=True)
finalData['MonthlyDebt'] = np.where(finalData['MonthlyDebt']==0, finalData['DebtRatio'], finalData['MonthlyDebt'])

finalData['isRetired'] = np.where((finalData['age'] > 65), 1, 0)

finalData['RevolvingLines'] = finalData['NumberOfOpenCreditLinesAndLoans']-finalData['NumberRealEstateLoansOrLines']
finalData['hasRevolvingLines']=np.where((finalData['RevolvingLines']>0),1,0)

finalData['hasMultipleRealEstates'] = np.where((finalData['NumberRealEstateLoansOrLines']>=2),1,0)

finalData['incomeDivByThousand'] = finalData['MonthlyIncome']/1000

In [ ]:
finalData

In [ ]:
missingvalues(finalData)

Bây giờ chúng tôi đã thêm các tính năng mới vào tập dữ liệu của mình. Tiếp theo, chúng tôi sẽ thực hiện kiểm tra độ lệch trên dữ liệu của mình bằng cách phân tích sự phân bố của các cột riêng lẻ và thực hiện Chuyển đổi Box Cox để giảm độ lệch.

## Kiểm tra độ lệch và chuyển đổi Box Cox

In [ ]:
columnList = list(finalData.columns)
columnList

fig = plt.figure(figsize=[25,25])
for col,i in zip(columnList,range(1,19)):
    axes = fig.add_subplot(6,3,i)
    sns.distplot(finalData[col],ax=axes, kde_kws={'bw':1.5}, color='red')
plt.show()

Từ các biểu đồ phân phối ở trên, chúng ta có thể thấy rằng phần lớn dữ liệu của chúng ta bị lệch theo một trong hai hướng. Chúng ta chỉ có thể thấy Tuổi hình thành gần với phân phối chuẩn. Hãy kiểm tra các giá trị độ lệch của mỗi cột

In [ ]:
def SkewMeasure(df):
    nonObjectColList = df.dtypes[df.dtypes != 'object'].index
    skewM = df[nonObjectColList].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending = False)
    skewM=pd.DataFrame({'skew':skewM})
    return skewM[abs(skewM)>0.5].dropna()

skewM = SkewMeasure(finalData)
skewM

Độ lệch rất cao đối với tất cả các cột. Chúng tôi sẽ áp dụng Biến đổi Box Cox với λ = 0,15 để giảm độ lệch này.

In [ ]:
for i in skewM.index:
    finalData[i] = special.boxcox1p(finalData[i],0.15) #lambda = 0.15
    
SkewMeasure(finalData)

Độ lệch đã giảm ở quy mô cao hơn nhiều khi mà Chuyển đổi Box Cox được áp dụng. Hãy kiểm tra lại các ô phân phối cho các cột riêng lẻ:

In [ ]:
fig = plt.figure(figsize=[25,25])
for col,i in zip(columnList,range(1,19)):
    axes = fig.add_subplot(6,3,i)
    sns.distplot(finalData[col],ax=axes, kde_kws={'bw':1.5}, color='blue')
plt.show()

## Mô Hình

Sẽ chia train và val thành tỷ lệ 70-30. Chia tập final ra thành trainDF với 149431 mẫu và 17 feature và testDF với 101503 mẫu với 17 feature.

In [ ]:
trainDF = finalData[:len(finalTrain)]
testDF = finalData[len(finalTrain):]
print(trainDF.shape)
print(testDF.shape)

In [ ]:
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(trainDF.to_numpy(),SeriousDlqIn2Yrs01.to_numpy(),test_size=0.3,random_state=2020)

## Decision tree classification and regression

In [ ]:
features = ['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse',
            'NumberOfTimes90DaysLate','DebtRatio','MonthlyIncome','DebtRatio','MonthlyIncomePerPerson','NumberOfOpenCreditLinesAndLoans',
            'NumberRealEstateLoansOrLines','NumberOfDependents','RevolvingLines','incomeDivByThousand']
x=trainDF[features]
x1=testDF[features]
#Chuẩn hóa dữ liệu
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(x.to_numpy(),SeriousDlqIn2Yrs01.to_numpy(),test_size=0.3,random_state=2020)
#tạo SMOTE
smote = SMOTE(random_state=2)
#tách dữ liệu
xtrain_resampled, ytrain_resampled = smote.fit_resample(xtrain, ytrain)

In [ ]:
start = datetime.datetime.now()
DTClassifier = DecisionTreeClassifier()
DTClassifier.fit(xtrain_resampled,ytrain_resampled)
end = datetime.datetime.now()
print("\nThời gian chạy trên mô hình Decision Tree là:", end - start)

In [ ]:
ypred = DTClassifier.predict(xtest)

In [ ]:
fprDTC, tprDTC, thresholdsDTC = metrics.roc_curve(y_true=ytest, y_score=ypred)
plt.figure(figsize=(12,6))
plt.title('ROC Curve')
sns.lineplot(fprDTC, tprDTC, label="AUC for Decision Tree Classifier Model (area = %.3f)" % metrics.auc(fprDTC, tprDTC))
plt.plot([0,1], [0,1], linestyle='--', label='random')
plt.plot([0,0,1], [0,1,1], linestyle='--', label="ideal")
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel("false positive rate")
plt.ylabel("true positive rate")

Cho thấy tỷ lẻ TP với FP và AUC bằng 0.614, tức là có 61,4% nhãn 1 được dự đoán đúng.

In [ ]:
skplt.metrics.plot_confusion_matrix(ytest,ypred,figsize=(6,6))
print("Test Accuracy:",DTClassifier.score(xtest, ytest))

In [ ]:
print(metrics.classification_report(ytest, ypred))
print("Accuracy:",metrics.accuracy_score(ytest, ypred))

Mô hình DTC có f1-score thấp và cả recall và precision đều thấp vì vậy mô hình này không dự đoán được nhiều label 1 thật. Mô hình bị overfit

In [ ]:
DecisionTreeClassifierMetrics = pd.DataFrame({'Model': 'DecisionTreeClassifier', 
                            'MSE': round(metrics.mean_squared_error(ytest, ypred)*100,2),
                            'RMSE' : round(np.sqrt(metrics.mean_squared_error(ytest, ypred)*100),2),
                            'MAE' : round(metrics.mean_absolute_error(ytest, ypred)*100,2),
                            'MSLE' : round(metrics.mean_squared_log_error(ytest, ypred)*100,2), 
                            'RMSLE' : round(np.sqrt(metrics.mean_squared_log_error(ytest, ypred)*100),2),
                            'Accuracy Train' : round(DTClassifier.score(xtrain, ytrain) * 100,2),
                            'Accuracy Test' : round(DTClassifier.score(xtest, ytest) * 100,2),
                            'F-Beta Score (β=2)' : round(metrics.fbeta_score(ytest, ypred, beta=2)*100,2)},index=[1])

DecisionTreeClassifierMetrics

In [ ]:
features = ['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse',
            'NumberOfTimes90DaysLate','DebtRatio','MonthlyIncome','DebtRatio','MonthlyIncomePerPerson','NumberOfOpenCreditLinesAndLoans',
            'NumberRealEstateLoansOrLines','NumberOfDependents','RevolvingLines','incomeDivByThousand']
x=trainDF[features]
x1=testDF[features]
#Chuẩn hóa dữ liệu
x_train, x_test, y_train, y_test = model_selection.train_test_split(x.to_numpy(),SeriousDlqIn2Yrs01.to_numpy(),test_size=0.3,random_state=2020)
# tạo StandardScaler và fit_trainsform
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.fit_transform(x_test)
#tạo SMOTE
smote = SMOTE(sampling_strategy="auto", random_state=10)
#tách dữ liệu
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_std, y_train)

In [ ]:
param_range = [0.001, 0.01, 0.1, 1.0, 10, 100]
penalty = ['l1', 'l2']
param_grid = [{"C":param_range, "penalty":penalty}]
start = datetime.datetime.now()
logmodel = LogisticRegression()
gs = GridSearchCV(estimator=logmodel, param_grid=param_grid, scoring="recall", cv=10, n_jobs=-1)
gs = gs.fit(x_train_resampled, y_train_resampled)
end = datetime.datetime.now()
print(gs.best_score_.round(3))
print(gs.best_params_)
print("\nThời gian chạy trên mô hình Logistic Regression là:", end - start)

In [ ]:
bestEstimatorGS= gs.best_estimator_
print('Test accuracy: %.3f' % bestEstimatorGS.score(x_test_std, y_test))

In [ ]:
y_pred = bestEstimatorGS.predict(x_test_std)
y_pred_train = bestEstimatorGS.predict(x_train_std)
#Xác nhận mô hình
print("*accuracy_train = %.3f" % metrics.accuracy_score(y_true=y_train, y_pred=y_pred_train))
print("accuracy = %.3f" % metrics.accuracy_score(y_true=y_test, y_pred=y_pred))

print("*precision_train = %.3f" % metrics.precision_score(y_true=y_train, y_pred=y_pred_train))
print("precision = %.3f" % metrics.precision_score(y_true=y_test, y_pred=y_pred))

print("*recall_train = %.3f" % metrics.recall_score(y_true=y_train, y_pred=y_pred_train))
print("recall = %.3f" % metrics.recall_score(y_true=y_test, y_pred=y_pred))

print("*f1_score_train = %.3f" % metrics.f1_score(y_true=y_train, y_pred=y_pred_train))
print("f1_score = %.3f" % metrics.f1_score(y_true=y_test, y_pred=y_pred))

In [ ]:
# ROC curve và AUC
y_score = bestEstimatorGS.predict_proba(x_test_std)[:, 1]

fprLR, tprLR, thresholdsLR = metrics.roc_curve(y_true=y_test, y_score=y_score)
# Visualization
plt.figure(figsize=(12,6))
plt.title('ROC Curve')
sns.lineplot(fprLR, tprLR, label="AUC for Logistic Regression model (area = %.3f)" % metrics.auc(fprLR, tprLR))
plt.plot([0,1], [0,1], linestyle='--', label='random')
plt.plot([0,0,1], [0,1,1], linestyle='--', label="ideal")
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel("false positive rate")
plt.ylabel("true positive rate")

Cho thấy tỷ lẻ TP với FP và AUC bằng 0.859, tức là có 85,9% nhãn 1 được dự đoán đúng

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test,y_pred,figsize=(6,6))

In [ ]:
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Độ chính xác accuracy bằng 0.80 khá cao nhưng trong tập dữ liệu mất cân bằng chúng ta quan tâm đến f1-score hay recall hay precision nếu TP hiếm hay label 1 hiếm. Precision khác thấp điều đó cho thấy các mẫu được dự đoán là 1 không phải là label 1 thật và recall bằng 0.74 khá cao cho thấy dự đoán được nhiều label 1

In [ ]:
LRegressionMetrics = pd.DataFrame({'Model': 'LogisticRegression', 
                            'MSE': round(metrics.mean_squared_error(y_test, y_pred)*100,2),
                            'RMSE' : round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)*100),2),
                            'MAE' : round(metrics.mean_absolute_error(y_test, y_pred)*100,2),
                            'MSLE' : round(metrics.mean_squared_log_error(y_test, y_pred)*100,2), 
                            'RMSLE' : round(np.sqrt(metrics.mean_squared_log_error(y_test, y_pred)*100),2),
                            'Accuracy Train' : round(bestEstimatorGS.score(x_train, y_train) * 100,2),     
                            'Accuracy Test' : round(bestEstimatorGS.score(x_test, y_test) * 100,2),
                            'F-Beta Score (β=2)' : round(metrics.fbeta_score(y_test, y_pred, beta=2)*100,2)},index=[1])

LRegressionMetrics

## LightGBM

Điều chỉnh siêu tham số

In [ ]:
lgbAttributes = lgb.LGBMClassifier(objective='binary', n_jobs=-1, random_state=2020, importance_type='gain')

lgbParameters = {
    'max_depth' : [2,3,4,5],
    'learning_rate': [0.05, 0.1,0.125,0.15],
    'colsample_bytree' : [0.2,0.4,0.6,0.8,1],
    'n_estimators' : [400,500,600,700,800,900],
    'min_split_gain' : [0.15,0.20,0.25,0.3,0.35], 
    'subsample': [0.6,0.7,0.8,0.9,1],
    'min_child_weight': [6,7,8,9,10],
    'scale_pos_weight': [10,15,20],
    'min_data_in_leaf' : [100,200,300,400,500,600,700,800,900],
    'num_leaves' : [20,30,40,50,60,70,80,90,100]
}

lgbModel = model_selection.RandomizedSearchCV(lgbAttributes, param_distributions = lgbParameters, cv = 5, random_state=2020)

lgbModel.fit(xTrain,yTrain.flatten(),feature_name=trainDF.columns.to_list())

In [ ]:
bestEstimatorLGB = lgbModel.best_estimator_
bestEstimatorLGB

Lưu công cụ ước tính tốt nhất từ RandomSearchCV

In [ ]:
start = datetime.datetime.now()
bestEstimatorLGB = lgb.LGBMClassifier(colsample_bytree=0.4, importance_type='gain', max_depth=5,
               min_child_weight=6, min_data_in_leaf=600, min_split_gain=0.25,
               n_estimators=900, num_leaves=50, objective='binary',
               random_state=2020, scale_pos_weight=10, subsample=0.9).fit(xTrain,yTrain.flatten(),feature_name=trainDF.columns.to_list())
end = datetime.datetime.now()
print("\nThời gian chạy trên mô hình Lgbm: ", end-start)

In [ ]:
yPredLGB = bestEstimatorLGB.predict_proba(xTest)
yPredLGB = yPredLGB[:,1]

In [ ]:
skplt.metrics.plot_confusion_matrix(yTest,yTestPredLGB,figsize=(6,6))

In [ ]:
yTestPredLGB = bestEstimatorLGB.predict(xTest)
print(metrics.classification_report(yTest,yTestPredLGB))

Độ chính xác accuracy bằng 0.86 khá cao nhưng trong tập dữ liệu mất cân bằng chúng ta quan tâm đến f1-score hay recall hay precision nếu TP hiếm hay label 1 hiếm. Precision khác thấp điều đó cho thấy các mẫu được dự đoán là 1 không phải là label 1 thật và recall bằng 0.67 khá cao cho thấy dự đoán được nhiều label 1. Sau đây là ma trận nhầm lẫn sẽ chỉ rõ điều đó hơn.

In [ ]:
LGBMMetrics = pd.DataFrame({'Model': 'LightGBM', 
                            'MSE': round(metrics.mean_squared_error(yTest, yTestPredLGB)*100,2),
                            'RMSE' : round(np.sqrt(metrics.mean_squared_error(yTest, yTestPredLGB)*100),2),
                            'MAE' : round(metrics.mean_absolute_error(yTest, yTestPredLGB)*100,2),
                            'MSLE' : round(metrics.mean_squared_log_error(yTest, yTestPredLGB)*100,2), 
                            'RMSLE' : round(np.sqrt(metrics.mean_squared_log_error(yTest, yTestPredLGB)*100),2),
                            'Accuracy Train' : round(bestEstimatorLGB.score(xTrain, yTrain) * 100,2),
                            'Accuracy Test' : round(bestEstimatorLGB.score(xTest, yTest) * 100,2),
                            'F-Beta Score (β=2)' : round(metrics.fbeta_score(yTest, yTestPredLGB, beta=2)*100,2)},index=[1])

LGBMMetrics

## ROC AUC

In [ ]:
fprLGBM, tprLGBM, thresholdsLGBM = metrics.roc_curve(yTest,yPredLGB)
rocAuc = metrics.auc(fprLGBM, tprLGBM)
plt.figure(figsize=(12,6))
plt.title('ROC Curve')
sns.lineplot(fprLGBM, tprLGBM, label = 'AUC for LightGBM Model (area = %.3f)' % rocAuc)
plt.plot([0,1], [0,1], linestyle='--', label='random')
plt.plot([0,0,1], [0,1,1], linestyle='--', label="ideal")
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Cho thấy tỷ lẻ TP với FP và AUC bằng 0.862, tức là có 86,2% nhãn 1 được dự đoán đúng

## FEATURE Quan Trọng

In [ ]:
lgb.plot_importance(bestEstimatorLGB, importance_type='gain')

In [ ]:
X = pd.DataFrame(xTrain, columns=trainDF.columns.to_list())

explainer = shap.TreeExplainer(bestEstimatorLGB)
shap_values = explainer.shap_values(X)

shap.summary_plot(shap_values[1], X)

## SO SÁNH CÁC MÔ HÌNH

In [ ]:
# Visualization
plt.figure(figsize=(12,6))
plt.title('ROC Curve')
sns.lineplot(fprLGBM, tprLGBM, label = 'AUC for LightGBM Model (area = %.3f)' % rocAuc, color='green')
sns.lineplot(fprDTC, tprDTC, label="AUC for Decision Tree Classifier Model (area = %.3f)" % metrics.auc(fprDTC, tprDTC), color='pink')
sns.lineplot(fprLR, tprLR, label="AUC for Logistic Regression model (area = %.3f)" % metrics.auc(fprLR, tprLR), color='yellow')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel("false positive rate")
plt.ylabel("true positive rate")

In [ ]:
frames = [LGBMMetrics, DecisionTreeClassifierMetrics, LRegressionMetrics]
TrainingResult = pd.concat(frames)
TrainingResult.T

## Kết quả

In [ ]:
lgbProbs = bestEstimatorLGB.predict_proba(testDF)
lgbDF = pd.DataFrame({'ID': testIndex, 'Probability': lgbProbs[:,1]})
lgbDF.to_csv('submissionLGBM.csv', index=False)

In [ ]:
lrProbs = bestEstimatorGS.predict_proba(x1)
lrDF = pd.DataFrame({'ID': testIndex, 'Probability': lrProbs[:,1]})
lrDF.to_csv('submissionLR.csv', index=False)